In [2]:
!pip install praw transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [3]:
import praw
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [4]:
##JYbYb9zkYmjX0S_REj5wYA
##UHIj7qN1myknVBpvMY9izU_DY5u3rA - key

In [5]:
REDDIT_CLIENT_ID = "JYbYb9zkYmjX0S_REj5wYA"
REDDIT_CLIENT_SECRET = "UHIj7qN1myknVBpvMY9izU_DY5u3rA"
REDDIT_USER_AGENT = "script:projectparse:v1.0 (by /u/Either-Remote-2697)"
REDDIT_USERNAME = "Hungry-Move-6603"  # Target user

In [6]:
reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID,
                     client_secret=REDDIT_CLIENT_SECRET,
                     user_agent=REDDIT_USER_AGENT)

In [7]:
def scrape_reddit_user(username, max_items=100):
    user = reddit.redditor(username)
    data = {
        "comments": [],
        "posts": [],
        "metadata": {
            "username": username,
            "created_utc": user.created_utc,
            "link_karma": user.link_karma,
            "comment_karma": user.comment_karma,
        }
    }

    try:
        for comment in user.comments.new(limit=max_items):
            data["comments"].append(f"[{comment.subreddit}] {comment.body}")

        for submission in user.submissions.new(limit=max_items):
            text = f"[{submission.subreddit}] {submission.title} - {submission.selftext}"
            data["posts"].append(text.strip())
    except Exception as e:
        print("Error fetching data:", e)

    return data

In [ ]:
HF_MODEL =  "tiiuae/falcon-7b-instruct" # can Change to mistral

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
def extract_username(profile_url):
    match = re.search(r"reddit\.com\/user\/([^\/]+)", profile_url)
    return match.group(1) if match else profile_url

# ========== Step 5: Scrape Reddit Data ==========
def scrape_reddit_user(username, max_items=100):
    user = reddit.redditor(username)
    data = {
        "comments": [],
        "posts": [],
        "metadata": {
            "username": username,
            "created_utc": user.created_utc,
            "link_karma": user.link_karma,
            "comment_karma": user.comment_karma,
        }
    }

    try:
        for comment in user.comments.new(limit=max_items):
            data["comments"].append(f"[{comment.subreddit}] {comment.body}")

        for submission in user.submissions.new(limit=max_items):
            text = f"[{submission.subreddit}] {submission.title} - {submission.selftext}"
            data["posts"].append(text.strip())
    except Exception as e:
        print("Error fetching data:", e)

    return data

In [18]:
def make_prompt(user_data):
    prompt = f"""
You are a professional UX researcher.

Your job is to generate a structured **user persona** for a Reddit user, based on their public posts and comments.

Use the format below, and for **every trait**, include a citation from one of their posts or comments.

---

## 🧑 Identity
(age, gender, region, profession, relationship status)
→ Add 1–2 citations from their content to justify.

## 💬 Personality
Describe tone, mindset, attitude.
→ Add 1–2 citations showing how they express themselves.

## 🎯 Motivations
What do they seem to care about? What drives them?
→ Cite specific comments/posts that reveal this.

## 📈 Behavior & Habits
How do they use Reddit? What topics? How often?
→ Add citations showing behavior patterns.

## 😤 Frustrations
What annoys or upsets them?
→ Add real examples.

## 🥅 Goals & Needs
What do they seem to want in life?
→ Use their posts to infer this.

## 🔍 Supporting Quotes
List a few raw quotes that support your analysis.

---

Here is the user's Reddit content:
"""

    for c in user_data["comments"][:10]:
        prompt += f"- {c}\n"
    for p in user_data["posts"][:5]:
        prompt += f"- {p}\n"

    prompt += "\nNow generate the full persona with citations as described."

    return prompt

In [19]:
def generate_persona(profile_url):
    username = extract_username(profile_url)
    user_data = scrape_reddit_user(username)
    prompt = make_prompt(user_data)

    result = generator(prompt, max_new_tokens=1024, do_sample=False, temperature=0.7)[0]['generated_text']

    filename = f"{username}_persona.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result)

    print(f"\n🎭 Persona saved to: {filename}")
    print("\n--- Preview ---\n")
    print(result[:1500] + "\n...\n")


In [20]:
import re

In [21]:
if __name__ == "__main__":
    # Paste any Reddit user profile URL below
    target_url = "https://www.reddit.com/user/Hungry-Move-6603/"
    generate_persona(target_url)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 186.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 30.12 MiB is free. Process 15397 has 14.71 GiB memory in use. Of the allocated memory 13.83 GiB is allocated by PyTorch, and 769.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [14]:
from google.colab import files
files.download("Hungry-Move-6603_persona.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

# Load text from file
with open("Hungry-Move-6603_persona_polished.txt", "r", encoding="utf-8") as f:
    persona_text = f.read()

# Setup image
img_width = 1080
margin = 40
font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"  # Adjust if on Windows

font_title = ImageFont.truetype(font_path, 36)
font_body = ImageFont.truetype(font_path, 24)

# Wrap and prepare text
def wrap_text(text, font, max_width):
    lines = []
    for paragraph in text.split('\n'):
        wrapped = textwrap.wrap(paragraph, width=70)
        lines.extend(wrapped if wrapped else [''])
    return lines

lines = wrap_text(persona_text, font_body, img_width - 2 * margin)

# Estimate image height
line_height = font_body.getsize("A")[1] + 8
img_height = line_height * len(lines) + 2 * margin

# Create image
img = Image.new("RGB", (img_width, img_height), color="white")
draw = ImageDraw.Draw(img)

# Draw text
y = margin
for line in lines:
    if line.startswith("##"):
        draw.text((margin, y), line.replace("##", "").strip(), font=font_title, fill="black")
        y += font_title.getsize("A")[1] + 16
    else:
        draw.text((margin, y), line, font=font_body, fill="gray")
        y += line_height

# Save image
output_file = "Hungry-Move-6603_persona_visual.png"
img.save(output_file)
print(f"✅ Persona image saved as: {output_file}")